The code to process Adult Census data, edit/train models, and perform adversarial debiasing. 

Necessary libraries for the notebook.

In [9]:
import os
import tensorflow as tf
import tf2onnx
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras import layers, models
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.utils import to_categorical
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.onnx
from scipy.io import savemat
import csv

### Data Preprocessing

In [2]:
def load_bank():
    # Define paths and column names
    file_path = '../data/bank/bank-additional-full.csv'
    column_names = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 
                    'month', 'day_of_week', 'duration', 'emp.var.rate', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
    na_values = ['unknown']
    
    # Load data
    df = pd.read_csv(file_path, sep=';', na_values=na_values)
    
    # Drop na values
    df.dropna(inplace=True)
    
    # Encode categorical features
    categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
    for col in categorical_features:
        df[col] = LabelEncoder().fit_transform(df[col])
    
    # Binary transformation for age (1 if age >= 25, else 0)
    df['age'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)
    
    # Convert target variable to binary
    df['y'] = df['y'].apply(lambda x: 1 if x == 'yes' else 0)
    
    # Select columns to keep (including the target variable 'y')
    df = df[column_names]
    
    # Split features and labels
    X = df.drop('y', axis=1)
    y = df['y']
    
    # Normalize features
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    
    # Extract the protected attribute ('age' for demonstration, adjust as needed)
    protected_attribute = X[:, df.columns.get_loc('age')]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test, protected_train, protected_test = train_test_split(
        X, to_categorical(y, num_classes=2), protected_attribute, test_size=0.2, random_state=42
    )
    
    return X_train, X_test, y_train, y_test, protected_train, protected_test

# Saves data for use in verification
def load_and_save_bank_data():
    X_train, X_test, y_train, y_test, _, _ = load_bank()
    
    # Scaling numerical features with MinMaxScaler
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Prepare data dictionary to save as .mat file
    data_dict = {
        'X': X_test, 
        'y': y_test   
    }
    
    # Save to .mat file for use in MATLAB
    savemat("./processed_data/bank_data.mat", data_dict)
    print("Data saved to bank_data.mat")

    return X_train, X_test, y_train, y_test

### Model Editing

Method to save the models as onnx files for verification. 

In [3]:
# Function to save the model as ONNX format
def save_model_onnx(model, input_shape, onnx_file_path):
    # Create a dummy input tensor with the correct input shape (batch_size, input_shape)
    dummy_input = tf.random.normal([1] + list(input_shape))

    # Convert the model to ONNX
    model_proto, external_tensor_storage = tf2onnx.convert.from_keras(model, 
                                                                      input_signature=(tf.TensorSpec(shape=[None] + list(input_shape), dtype=tf.float32),),
                                                                      opset=13)
    
    # Save the ONNX model to the specified path
    with open(onnx_file_path, "wb") as f:
        f.write(model_proto.SerializeToString())
    
    print(f"Model has been saved in ONNX format at {onnx_file_path}")

Change the models so they are able to be used in FairNNV. FairNNV cannot handle sigmoid so shift to softmax and adjust final layers. 

In [14]:
# Function to modify a model for multiclass classification
def modify_model_for_multiclass(model_path, num_classes):
    # with warnings.catch_warnings():
    #     warnings.simplefilter("ignore", category=UserWarning)
    model = load_model(model_path)

    # Create a new input layer with the correct shape
    new_input = tf.keras.layers.Input(shape=(16,))
    x = new_input

    # Transfer the layers except the last one
    for layer in model.layers[:-1]:
        x = layer(x)

    # Create a new output layer
    output = tf.keras.layers.Dense(num_classes, activation='softmax', name='new_output')(x)
    
    # Create a new model
    new_model = tf.keras.models.Model(inputs=new_input, outputs=output)
    
    return new_model

# Ensure the save directories exist
model_dir = './bank/bank_h5'
save_dir = './bank/bank_keras'
onnx_save_dir = './bank/bank_onnx'
num_classes = 2

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
if not os.path.exists(onnx_save_dir):
    os.makedirs(onnx_save_dir)

# Modify each model in the directory to remove sigmoid
for model_file in os.listdir(model_dir):
    if model_file.endswith('.h5'):
        model_path = os.path.join(model_dir, model_file)
        new_model = modify_model_for_multiclass(model_path, num_classes)
        
        # Update the model's loss function
        new_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        
        # Save the modified model
        save_path = os.path.join(save_dir, model_file.replace('.h5', '.keras'))
        new_model.save(save_path)


/home/annemtumlin/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/annemtumlin/.local/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Re-train models. 

In [15]:
# Load and preprocess the Bank dataset
X_train, X_test, y_train, y_test = load_and_save_bank_data()

for model_file in os.listdir(save_dir):
    if model_file.endswith('.keras'):
        model_path = os.path.join(save_dir, model_file)
        
        try:
            # Load the modified model
            print(f"Loading model {model_file}")
            # with warnings.catch_warnings():
            #     warnings.simplefilter("ignore", category=UserWarning)
            model = load_model(model_path)

            # Reinitialize the optimizer
            model.compile(
                optimizer=Adam(),
                loss='categorical_crossentropy', 
                metrics=['accuracy']
            )

            # Fit the model
            print(f"Training model {model_file}")
            history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

            # Evaluate the model
            y_pred = model.predict(X_test)
            y_pred_classes = np.argmax(y_pred, axis=1)
            accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_classes)

            print(f"Model {model_file} - Accuracy: {accuracy}")

            # Save the retrained model
            model.save(model_path)
            print(f"Model {model_file} retrained and saved successfully.")

            # Save the model as ONNX
            onnx_save_path = os.path.join(onnx_save_dir, model_file.replace('.keras', '.onnx'))
            save_model_onnx(model, (16,), onnx_save_path)

        except Exception as e:
            print(f"Failed to process {model_file}. Error: {e}")

Data saved to bank_data.mat
Loading model BM-7.keras
Training model BM-7.keras
Epoch 1/50


/home/annemtumlin/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


610/610 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8619 - loss: 0.3433 - val_accuracy: 0.8784 - val_loss: 0.3261
Epoch 2/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8806 - loss: 0.3161 - val_accuracy: 0.8831 - val_loss: 0.2977
Epoch 3/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8929 - loss: 0.2735 - val_accuracy: 0.8872 - val_loss: 0.2645
Epoch 4/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8927 - loss: 0.2519 - val_accuracy: 0.8913 - val_loss: 0.2511
Epoch 5/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9012 - loss: 0.2335 - val_accuracy: 0.8899 - val_loss: 0.2492
Epoch 6/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8999 - loss: 0.2295 - val_accuracy: 0.8899 - val_loss: 0.2496
Epoch 7/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9009 - loss: 0.2272 - val_accuracy: 0.8879 - val_loss: 0.2556
Epoch 8/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9005 - loss: 0.2262 - val_accuracy: 0.8932 - val_

2024-07-19 15:53:01.129250: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-19 15:53:01.129352: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-19 15:53:01.151177: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-19 15:53:01.151320: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
/home/annemtumlin/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8723 - loss: 0.5038 - val_accuracy: 0.8713 - val_loss: 0.3396
Epoch 2/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8722 - loss: 0.3325 - val_accuracy: 0.8713 - val_loss: 0.3324
Epoch 3/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8757 - loss: 0.3247 - val_accuracy: 0.8694 - val_loss: 0.3265
Epoch 4/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8731 - loss: 0.3227 - val_accuracy: 0.8700 - val_loss: 0.3235
Epoch 5/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8742 - loss: 0.3179 - val_accuracy: 0.8774 - val_loss: 0.3171
Epoch 6/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8789 - loss: 0.3156 - val_accuracy: 0.8784 - val_loss: 0.3127
Epoch 7/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8824 - loss: 0.3060 - val_accuracy: 0.8823 - val_loss: 0.3095
Epoch 8/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8844 - loss: 0.3022 - val_accuracy: 0.8829 - val_

...

2024-07-19 15:56:20.287058: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-19 15:56:20.287149: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-19 15:56:20.306438: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-19 15:56:20.306528: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
/home/annemtumlin/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


610/610 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8406 - loss: 0.6006 - val_accuracy: 0.8676 - val_loss: 0.3525
Epoch 2/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8765 - loss: 0.3296 - val_accuracy: 0.8797 - val_loss: 0.2976
Epoch 3/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8851 - loss: 0.2897 - val_accuracy: 0.8854 - val_loss: 0.2814
Epoch 4/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8887 - loss: 0.2714 - val_accuracy: 0.8852 - val_loss: 0.2789
Epoch 5/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8900 - loss: 0.2608 - val_accuracy: 0.8883 - val_loss: 0.2584
Epoch 6/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8947 - loss: 0.2511 - val_accuracy: 0.8897 - val_loss: 0.2570
Epoch 7/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8997 - loss: 0.2380 - val_accuracy: 0.8907 - val_loss: 0.2533
Epoch 8/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8903 - loss: 0.2487 - val_accuracy: 0.8893 - val_

2024-07-19 15:56:57.261902: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-19 15:56:57.261989: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-19 15:56:57.282848: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-19 15:56:57.282965: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
/home/annemtumlin/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8654 - loss: 2.1468 - val_accuracy: 0.8692 - val_loss: 0.3443
Epoch 2/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8722 - loss: 0.3309 - val_accuracy: 0.8711 - val_loss: 0.3337
Epoch 3/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8762 - loss: 0.3210 - val_accuracy: 0.8729 - val_loss: 0.3282
Epoch 4/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8829 - loss: 0.3106 - val_accuracy: 0.8745 - val_loss: 0.3195
Epoch 5/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8797 - loss: 0.3073 - val_accuracy: 0.8793 - val_loss: 0.3069
Epoch 6/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8880 - loss: 0.2880 - val_accuracy: 0.8838 - val_loss: 0.2779
Epoch 7/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8877 - loss: 0.2678 - val_accuracy: 0.8881 - val_loss: 0.2565
Epoch 8/50
610/610 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8930 - loss: 0.2405 - val_accuracy: 0.8823 - val_

2024-07-19 15:57:35.946426: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-19 15:57:35.946526: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-07-19 15:57:35.970757: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-07-19 15:57:35.970968: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
